## 1. Configuration & Setup

# HubSpot to Twenty CRM - Production Migration Script

## 📋 Overview
Complete migration pipeline for importing HubSpot data (Companies, Contacts, Deals) into Twenty CRM with:
- ✅ Automatic database schema preparation (ENUM values)
- ✅ Automatic column detection and creation for unmapped HubSpot fields
- ✅ Foreign key resolution with deduplication (domain → company, email → person)
- ✅ Data type conversion and validation
- ✅ Transaction safety with per-row savepoints
- ✅ 100% success rate on test data (33/33 records)

## 🚀 Quick Start
1. **Configure Mode**: Set `USE_TEST_DATA = True/False` in Section 1
2. **Run Sections 1-3b**: Load data, prepare schema, detect columns
3. **Run Section 5**: Execute migration
4. **Run Section 6**: Verify results
5. **Optional - Section 7**: Purge data for re-migration

## 📊 Migration Statistics (Latest Run)
- Companies: 13 ✅
- Contacts: 10 ✅
- Deals: 10 ✅
- Success Rate: 100%
- Relationships: 90% contacts linked to companies

In [6]:
# Import libraries
import pandas as pd
import psycopg2
import json
from pathlib import Path
from datetime import datetime
import hashlib
import uuid

In [35]:
# Database Configuration
TWENTY_DB_CONFIG = {
    'host': 'twentypsql.postgres.database.azure.com',
    'database': 'twenty2',
    'user': 'twentyadmin',
    'password': 'ISTARIAdmin!',
    'port': 5432
}

WORKSPACE_SCHEMA = 'workspace_bqt42ghwp3i4ag4dk3riyqxco'

# Data paths - SWITCH BETWEEN TEST AND FULL
USE_TEST_DATA = True  # Set to True for test migration

if USE_TEST_DATA:
    DATA_DIR = Path('migration_data_test')
    print("🧪 TEST MODE: Using test data files (10 companies, 7 contacts, 10 deals)")
else:
    DATA_DIR = Path('migration_data')
    print("🚀 PRODUCTION MODE: Using full data files (33,757 total records)")

CSV_FILES = {
    'companies': DATA_DIR / ('test-companies.csv' if USE_TEST_DATA else 'all-companies.csv'),
    'contacts': DATA_DIR / ('test-contacts.csv' if USE_TEST_DATA else 'all-contacts.csv'),
    'deals': DATA_DIR / ('test-deals.csv' if USE_TEST_DATA else 'all-deals.csv')
}

def get_db_connection(config):
    """Create database connection"""
    conn = psycopg2.connect(**config)
    print("✓ Database connection established")
    return conn

🧪 TEST MODE: Using test data files (10 companies, 7 contacts, 10 deals)


## 2. Load Data & Schema

In [36]:
# Load CSV files
print("Loading HubSpot data...")
companies_df = pd.read_csv(CSV_FILES['companies'], low_memory=False)
contacts_df = pd.read_csv(CSV_FILES['contacts'], low_memory=False)
deals_df = pd.read_csv(CSV_FILES['deals'], low_memory=False)

print(f"✅ Companies: {len(companies_df)} records × {len(companies_df.columns)} columns")
print(f"✅ Contacts:  {len(contacts_df)} records × {len(contacts_df.columns)} columns")
print(f"✅ Deals:     {len(deals_df)} records × {len(deals_df.columns)} columns")
print(f"\n📊 Total records to migrate: {len(companies_df) + len(contacts_df) + len(deals_df)}")

Loading HubSpot data...
✅ Companies: 13 records × 182 columns
✅ Contacts:  10 records × 311 columns
✅ Deals:     10 records × 146 columns

📊 Total records to migrate: 33


In [37]:
# Load Twenty schema (generated from database)
with open('twenty_actual_schema.json', 'r') as f:
    actual_schema = json.load(f)

# Create column name and type mappings
company_cols = {col['column_name'] for col in actual_schema['company']}
person_cols = {col['column_name'] for col in actual_schema['person']}
opportunity_cols = {col['column_name'] for col in actual_schema['opportunity']}

company_types = {col['column_name']: col['data_type'] for col in actual_schema['company']}
person_types = {col['column_name']: col['data_type'] for col in actual_schema['person']}
opportunity_types = {col['column_name']: col['data_type'] for col in actual_schema['opportunity']}

print(f"Twenty schema loaded:")
print(f"  Company: {len(company_cols)} columns")
print(f"  Person: {len(person_cols)} columns")
print(f"  Opportunity: {len(opportunity_cols)} columns")

Twenty schema loaded:
  Company: 204 columns
  Person: 327 columns
  Opportunity: 154 columns


## 3. Helper Functions

In [39]:
# Check unique Deal Stage values in the data
print("="*100)
print("UNIQUE DEAL STAGE VALUES IN TEST DATA")
print("="*100)
unique_stages = deals_df['Deal Stage'].dropna().unique()
print(f"\nFound {len(unique_stages)} unique stages:")
for stage in sorted(unique_stages):
    print(f"  - {stage}")

UNIQUE DEAL STAGE VALUES IN TEST DATA

Found 4 unique stages:
  - Exploration: Meeting scheduled
  - Platform Subscribed
  - Unfit
  - Unfit VIP


In [40]:
# Check current ENUM values in database
print("\n" + "="*100)
print("CHECKING CURRENT ENUM VALUES IN DATABASE")
print("="*100)

try:
    conn = get_db_connection(TWENTY_DB_CONFIG)
    cursor = conn.cursor()

    # Query to get ENUM type values
    cursor.execute("""
        SELECT
            t.typname AS enum_name,
            e.enumlabel AS enum_value,
            e.enumsortorder
        FROM pg_type t
        JOIN pg_enum e ON t.oid = e.enumtypid
        WHERE t.typname LIKE '%stage%'
        ORDER BY t.typname, e.enumsortorder
    """)

    results = cursor.fetchall()

    if results:
        current_enum = results[0][0]
        print(f"\nFound ENUM type: {current_enum}")
        print(f"\nCurrent values ({len(results)}):")
        for enum_name, enum_value, sort_order in results:
            print(f"  {sort_order}. {enum_value}")
    else:
        print("\n⚠️  No stage-related ENUM types found")

    cursor.close()
    conn.close()

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()


CHECKING CURRENT ENUM VALUES IN DATABASE
✓ Database connection established

Found ENUM type: opportunity_stage_enum

Current values (14):
  1.0. OUTREACH_LEADS_TO_CONTACT
  2.0. EXPLORATION_MEETING_SCHEDULED
  3.0. OFFER_OFFER_REQUESTED
  4.0. OFFER_OFFER_SENT
  5.0. SIGNED_ONE_OFF
  6.0. SIGNED_RECURRING
  7.0. LOST_VIP
  8.0. LOST_TRASH
  9.0. LOST_REGIONS
  10.0. OFFER_TENDER_SENT
  11.0. PLATFORM_CANCELED
  12.0. PLATFORM_SUBSCRIBED
  13.0. PROJECT_COMPLETED_AWAITING_FEEDBACK
  14.0. PROJECT_PAID


## 3a. Database Schema Preparation - Add Missing ENUM Values

In [41]:
def add_missing_enum_values():
    """Add new ENUM values to opportunity_stage_enum if they don't exist"""

    # Define all stage values we need (based on HubSpot data)
    required_stages = {
        'OUTREACH_LEADS_TO_CONTACT',
        'EXPLORATION_MEETING_SCHEDULED',
        'OFFER_OFFER_REQUESTED',
        'OFFER_OFFER_SENT',
        'OFFER_TENDER_SENT',
        'SIGNED_ONE_OFF',
        'SIGNED_RECURRING',
        'PROJECT_COMPLETED_AWAITING_FEEDBACK',
        'PROJECT_PAID',
        'PLATFORM_SUBSCRIBED',
        'PLATFORM_CANCELED',
        'LOST_VIP',
        'LOST_REGIONS',
        'LOST_TRASH'
    }

    try:
        conn = get_db_connection(TWENTY_DB_CONFIG)
        cursor = conn.cursor()

        # Get existing ENUM values
        cursor.execute("""
            SELECT e.enumlabel
            FROM pg_type t
            JOIN pg_enum e ON t.oid = e.enumtypid
            WHERE t.typname = 'opportunity_stage_enum'
        """)

        existing_stages = {row[0] for row in cursor.fetchall()}
        print(f"Existing ENUM values: {len(existing_stages)}")

        # Find missing stages
        missing_stages = required_stages - existing_stages

        if not missing_stages:
            print("✅ All required ENUM values already exist!")
            cursor.close()
            conn.close()
            return True

        print(f"\n📝 Adding {len(missing_stages)} new ENUM values:")

        # Add each missing stage
        for stage in sorted(missing_stages):
            try:
                print(f"  Adding: {stage}...")
                cursor.execute(f"""
                    ALTER TYPE {WORKSPACE_SCHEMA}.opportunity_stage_enum
                    ADD VALUE IF NOT EXISTS '{stage}'
                """)
                conn.commit()
                print(f"    ✅ Added")
            except Exception as e:
                print(f"    ⚠️  {str(e)[:100]}")
                conn.rollback()

        # Verify all values now exist
        cursor.execute("""
            SELECT e.enumlabel
            FROM pg_type t
            JOIN pg_enum e ON t.oid = e.enumtypid
            WHERE t.typname = 'opportunity_stage_enum'
            ORDER BY e.enumsortorder
        """)

        final_stages = [row[0] for row in cursor.fetchall()]
        print(f"\n✅ Final ENUM values: {len(final_stages)}")
        for i, stage in enumerate(final_stages, 1):
            print(f"  {i}. {stage}")

        cursor.close()
        conn.close()
        return True

    except Exception as e:
        print(f"\n❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return False

# Execute the function
print("="*100)
print("PREPARING DATABASE SCHEMA")
print("="*100)
add_missing_enum_values()

PREPARING DATABASE SCHEMA
✓ Database connection established
Existing ENUM values: 14
✅ All required ENUM values already exist!


True

## 3b. Column Detection & Schema Sync - Auto-detect Unmapped HubSpot Fields

In [42]:
def detect_and_add_unmapped_columns(auto_execute=False):
    """
    Detect unmapped columns from HubSpot CSV and optionally add them to database

    Args:
        auto_execute: If True, automatically execute ALTER TABLE statements
                      If False, only preview the changes
    """

    print("="*100)
    print("DETECTING UNMAPPED HUBSPOT COLUMNS")
    print("="*100)

    # Define column mappings for normalization
    def normalize_column_name(hubspot_col):
        """Convert HubSpot column name to PostgreSQL column name"""
        pg_col = hubspot_col.lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        pg_col = ''.join(c for c in pg_col if c.isalnum() or c == '_')
        return pg_col

    def infer_pg_type(series):
        """Infer PostgreSQL type from pandas Series"""
        # Skip if all null
        if series.isna().all():
            return 'text'

        # Get non-null values
        non_null = series.dropna()

        # Try to determine type
        if pd.api.types.is_integer_dtype(non_null):
            return 'bigint'
        elif pd.api.types.is_float_dtype(non_null):
            return 'double precision'
        elif pd.api.types.is_bool_dtype(non_null):
            return 'boolean'
        elif pd.api.types.is_datetime64_any_dtype(non_null):
            return 'timestamp with time zone'
        else:
            # Check string length for text vs varchar
            if non_null.astype(str).str.len().max() > 255:
                return 'text'
            else:
                return 'text'  # Use text for flexibility

    unmapped_columns = {
        'company': [],
        'person': [],
        'opportunity': []
    }

    # Skip columns that are already handled or internal
    skip_columns = {
        'Record ID', 'Create Date', 'Last Modified Date',
        'Company name', 'Website URL', 'Number of Employees', 'LinkedIn Company Page',
        'City', 'State/Region', 'Country/Region', 'Postal Code', 'Street Address',
        'First Name', 'Last Name', 'Email', 'Phone Number', 'Job Title', 'LinkedIn Profile',
        'Associated Company IDs', 'Associated Company IDs (Primary)',
        'Associated Contact IDs', 'Associated Contact IDs (Primary)',
        'Deal Name', 'Amount', 'Close Date', 'Deal Stage', 'Deal Currency Code'
    }

    # Check companies CSV
    print("\n📊 COMPANIES CSV:")
    print(f"   Total columns: {len(companies_df.columns)}")
    for hubspot_col in companies_df.columns:
        if hubspot_col in skip_columns:
            continue

        pg_col = normalize_column_name(hubspot_col)
        if pg_col not in company_cols:
            pg_type = infer_pg_type(companies_df[hubspot_col])
            unmapped_columns['company'].append({
                'hubspot_name': hubspot_col,
                'pg_name': pg_col,
                'pg_type': pg_type,
                'non_null_count': companies_df[hubspot_col].notna().sum(),
                'total_count': len(companies_df)
            })

    print(f"   Unmapped columns: {len(unmapped_columns['company'])}")
    for col in unmapped_columns['company'][:5]:  # Show first 5
        print(f"     - {col['hubspot_name']} → {col['pg_name']} ({col['pg_type']}) - {col['non_null_count']}/{col['total_count']} have data")
    if len(unmapped_columns['company']) > 5:
        print(f"     ... and {len(unmapped_columns['company']) - 5} more")

    # Check contacts CSV
    print("\n📊 CONTACTS CSV:")
    print(f"   Total columns: {len(contacts_df.columns)}")
    for hubspot_col in contacts_df.columns:
        if hubspot_col in skip_columns:
            continue

        pg_col = normalize_column_name(hubspot_col)
        if pg_col not in person_cols:
            pg_type = infer_pg_type(contacts_df[hubspot_col])
            unmapped_columns['person'].append({
                'hubspot_name': hubspot_col,
                'pg_name': pg_col,
                'pg_type': pg_type,
                'non_null_count': contacts_df[hubspot_col].notna().sum(),
                'total_count': len(contacts_df)
            })

    print(f"   Unmapped columns: {len(unmapped_columns['person'])}")
    for col in unmapped_columns['person'][:5]:
        print(f"     - {col['hubspot_name']} → {col['pg_name']} ({col['pg_type']}) - {col['non_null_count']}/{col['total_count']} have data")
    if len(unmapped_columns['person']) > 5:
        print(f"     ... and {len(unmapped_columns['person']) - 5} more")

    # Check deals CSV
    print("\n📊 DEALS CSV:")
    print(f"   Total columns: {len(deals_df.columns)}")
    for hubspot_col in deals_df.columns:
        if hubspot_col in skip_columns:
            continue

        pg_col = normalize_column_name(hubspot_col)
        if pg_col not in opportunity_cols:
            pg_type = infer_pg_type(deals_df[hubspot_col])
            unmapped_columns['opportunity'].append({
                'hubspot_name': hubspot_col,
                'pg_name': pg_col,
                'pg_type': pg_type,
                'non_null_count': deals_df[hubspot_col].notna().sum(),
                'total_count': len(deals_df)
            })

    print(f"   Unmapped columns: {len(unmapped_columns['opportunity'])}")
    for col in unmapped_columns['opportunity'][:5]:
        print(f"     - {col['hubspot_name']} → {col['pg_name']} ({col['pg_type']}) - {col['non_null_count']}/{col['total_count']} have data")
    if len(unmapped_columns['opportunity']) > 5:
        print(f"     ... and {len(unmapped_columns['opportunity']) - 5} more")

    # Generate ALTER TABLE statements
    total_unmapped = len(unmapped_columns['company']) + len(unmapped_columns['person']) + len(unmapped_columns['opportunity'])

    if total_unmapped == 0:
        print("\n✅ All HubSpot columns are already mapped to database schema!")
        return True

    print(f"\n\n{'='*100}")
    print(f"TOTAL UNMAPPED COLUMNS: {total_unmapped}")
    print(f"{'='*100}")

    alter_statements = []

    # Generate ALTER TABLE for company
    if unmapped_columns['company']:
        print(f"\n📝 ALTER TABLE statements for COMPANY ({len(unmapped_columns['company'])} columns):")
        for col in unmapped_columns['company']:
            stmt = f'ALTER TABLE {WORKSPACE_SCHEMA}.company ADD COLUMN IF NOT EXISTS "{col["pg_name"]}" {col["pg_type"]};'
            alter_statements.append(('company', stmt))
            print(f"   {stmt}")

    # Generate ALTER TABLE for person
    if unmapped_columns['person']:
        print(f"\n📝 ALTER TABLE statements for PERSON ({len(unmapped_columns['person'])} columns):")
        for col in unmapped_columns['person']:
            stmt = f'ALTER TABLE {WORKSPACE_SCHEMA}.person ADD COLUMN IF NOT EXISTS "{col["pg_name"]}" {col["pg_type"]};'
            alter_statements.append(('person', stmt))
            print(f"   {stmt}")

    # Generate ALTER TABLE for opportunity
    if unmapped_columns['opportunity']:
        print(f"\n📝 ALTER TABLE statements for OPPORTUNITY ({len(unmapped_columns['opportunity'])} columns):")
        for col in unmapped_columns['opportunity']:
            stmt = f'ALTER TABLE {WORKSPACE_SCHEMA}.opportunity ADD COLUMN IF NOT EXISTS "{col["pg_name"]}" {col["pg_type"]};'
            alter_statements.append(('opportunity', stmt))
            print(f"   {stmt}")

    # Execute if requested
    if auto_execute:
        print(f"\n\n{'='*100}")
        print("EXECUTING ALTER TABLE STATEMENTS")
        print(f"{'='*100}")

        try:
            conn = get_db_connection(TWENTY_DB_CONFIG)
            cursor = conn.cursor()

            success_count = 0
            for table, stmt in alter_statements:
                try:
                    cursor.execute(stmt)
                    conn.commit()
                    success_count += 1
                except Exception as e:
                    print(f"  ⚠️  Error on {table}: {str(e)[:100]}")
                    conn.rollback()

            print(f"\n✅ Successfully executed {success_count}/{len(alter_statements)} ALTER TABLE statements")

            # Update schema tracking in memory
            print("\n📝 Updating schema tracking...")
            for table, col_info in [('company', unmapped_columns['company']),
                                     ('person', unmapped_columns['person']),
                                     ('opportunity', unmapped_columns['opportunity'])]:
                for col in col_info:
                    if table == 'company':
                        company_cols.add(col['pg_name'])
                        company_types[col['pg_name']] = col['pg_type']
                    elif table == 'person':
                        person_cols.add(col['pg_name'])
                        person_types[col['pg_name']] = col['pg_type']
                    elif table == 'opportunity':
                        opportunity_cols.add(col['pg_name'])
                        opportunity_types[col['pg_name']] = col['pg_type']

            print(f"✅ Schema tracking updated:")
            print(f"   Company: {len(company_cols)} columns")
            print(f"   Person: {len(person_cols)} columns")
            print(f"   Opportunity: {len(opportunity_cols)} columns")

            cursor.close()
            conn.close()

            return True

        except Exception as e:
            print(f"\n❌ Error: {e}")
            import traceback
            traceback.print_exc()
            return False
    else:
        print(f"\n\n⚠️  PREVIEW MODE: Set auto_execute=True to add these columns to the database")
        return False

# Execute column detection
print("\n🔍 SCANNING FOR UNMAPPED HUBSPOT COLUMNS...")
detect_and_add_unmapped_columns(auto_execute=True)


🔍 SCANNING FOR UNMAPPED HUBSPOT COLUMNS...
DETECTING UNMAPPED HUBSPOT COLUMNS

📊 COMPANIES CSV:
   Total columns: 182
   Unmapped columns: 1
     - Company Domain Name → company_domain_name (text) - 13/13 have data

📊 CONTACTS CSV:
   Total columns: 311
   Unmapped columns: 1
     - LinkedIn URL → linkedin_url (text) - 0/10 have data

📊 DEALS CSV:
   Total columns: 146
   Unmapped columns: 0


TOTAL UNMAPPED COLUMNS: 2

📝 ALTER TABLE statements for COMPANY (1 columns):
   ALTER TABLE workspace_bqt42ghwp3i4ag4dk3riyqxco.company ADD COLUMN IF NOT EXISTS "company_domain_name" text;

📝 ALTER TABLE statements for PERSON (1 columns):
   ALTER TABLE workspace_bqt42ghwp3i4ag4dk3riyqxco.person ADD COLUMN IF NOT EXISTS "linkedin_url" text;


EXECUTING ALTER TABLE STATEMENTS
✓ Database connection established

✅ Successfully executed 2/2 ALTER TABLE statements

📝 Updating schema tracking...
✅ Schema tracking updated:
   Company: 205 columns
   Person: 328 columns
   Opportunity: 154 columns


True

In [ ]:
def generate_uuid_from_hubspot_id(hubspot_id):
    """Generate consistent UUID from HubSpot Record ID"""
    namespace = uuid.UUID('6ba7b810-9dad-11d1-80b4-00c04fd430c8')
    return str(uuid.uuid5(namespace, str(hubspot_id)))

def parse_timestamp(timestamp_str):
    """Parse HubSpot timestamp to ISO format"""
    if pd.isna(timestamp_str):
        return None
    try:
        dt = pd.to_datetime(timestamp_str)
        return dt.isoformat() + 'Z' if dt.tzinfo is None else dt.isoformat()
    except:
        return None

def map_deal_stage(hubspot_stage):
    """Map HubSpot deal stages to Twenty ENUM stage values"""
    stage_str = str(hubspot_stage)
    stage_lower = stage_str.lower()

    # Exact mapping: HubSpot stage → Twenty ENUM value
    stage_mapping = {
        'outreach: leads to contact': 'OUTREACH_LEADS_TO_CONTACT',
        'exploration: meeting scheduled': 'EXPLORATION_MEETING_SCHEDULED',
        'exploration: offer requested': 'OFFER_OFFER_REQUESTED',
        'exploration: offer sent': 'OFFER_OFFER_SENT',
        'exploration: tender sent': 'OFFER_OFFER_SENT',  # Maps to same as "Offer Sent"
        'project signed': 'SIGNED_ONE_OFF',
        'project completed: awaiting feedback': 'SIGNED_ONE_OFF',
        'project paid': 'SIGNED_ONE_OFF',
        'platform subscribed': 'SIGNED_RECURRING',
        'platform canceled': 'LOST_VIP',
        'unfit vip': 'LOST_VIP',
        'unfit regions': 'LOST_TRASH',
        'unfit': 'LOST_TRASH'
    }

    # Check exact match first (case-insensitive)
    if stage_lower in stage_mapping:
        return stage_mapping[stage_lower]

    # Fallback to default stage for unmapped values
    print(f"⚠️  Unmapped stage '{hubspot_stage}' → defaulting to OUTREACH_LEADS_TO_CONTACT")
    return 'OUTREACH_LEADS_TO_CONTACT'

def convert_value_to_target_type(value, target_type):
    """Convert value to match PostgreSQL data type"""
    if value is None or (isinstance(value, str) and value.strip() == ''):
        return None

    try:
        if target_type in ['numeric', 'double precision']:
            if isinstance(value, str):
                if value.lower() in ['true', 't', 'yes', 'y']:
                    return 1
                elif value.lower() in ['false', 'f', 'no', 'n']:
                    return 0
            return float(value)

        elif target_type in ['integer', 'bigint', 'smallint']:
            if isinstance(value, str):
                if value.lower() in ['true', 't', 'yes', 'y']:
                    return 1
                elif value.lower() in ['false', 'f', 'no', 'n']:
                    return 0
            int_value = int(float(value))
            # Skip values that would overflow PostgreSQL integer (32-bit)
            if target_type == 'integer' and abs(int_value) > 2147483647:
                return None
            return int_value

        elif target_type == 'boolean':
            if isinstance(value, bool):
                return value
            if isinstance(value, str):
                return value.lower() in ['true', 't', 'yes', 'y', '1']
            return bool(value)

        elif target_type in ['text', 'character varying', 'varchar']:
            return str(value)

        else:
            return value

    except (ValueError, TypeError):
        return None

print("✅ Helper functions loaded")

✅ Helper functions loaded


In [18]:
# Test the updated mapping with actual data
print("="*100)
print("TESTING UPDATED STAGE MAPPING")
print("="*100)

test_stages = deals_df['Deal Stage'].dropna().unique()
print(f"\nMapping {len(test_stages)} unique stages from test data:\n")

for stage in sorted(test_stages):
    mapped = map_deal_stage(stage)
    print(f"  '{stage}' → {mapped}")

TESTING UPDATED STAGE MAPPING

Mapping 4 unique stages from test data:

  'Exploration: Meeting scheduled' → EXPLORATION_MEETING_SCHEDULED
  'Platform Subscribed' → SIGNED_RECURRING
  'Unfit' → LOST_TRASH
  'Unfit VIP' → LOST_VIP


## 4. Data Transformation Functions

In [27]:
# Track inserted IDs for foreign key validation and cleanup
inserted_company_ids = set()
inserted_person_ids = set()

# NEW: FK resolution helpers - these will be populated during migration
domain_to_company_id = {}  # Maps domain → actual company.id from DB
email_to_person_id = {}     # Maps email → actual person.id from DB
hubspot_company_to_domain = {}  # Maps HubSpot company Record ID → domain
hubspot_contact_to_email = {}   # Maps HubSpot contact Record ID → email

def build_company_insert_data(row):
    """Build company insert data with type conversion"""
    data = {}

    # Core fields
    data['id'] = generate_uuid_from_hubspot_id(str(row['Record ID']))
    data['name'] = str(row.get('Company name', '')) if pd.notna(row.get('Company name')) else None

    if pd.notna(row.get('Website URL')):
        domain = str(row['Website URL'])
        data['domainNamePrimaryLinkUrl'] = domain
        # Track mapping for FK resolution
        hubspot_company_to_domain[str(row['Record ID'])] = domain

    if pd.notna(row.get('Number of Employees')):
        data['employees'] = convert_value_to_target_type(row['Number of Employees'], 'double precision')

    if pd.notna(row.get('LinkedIn Company Page')):
        data['linkedinLinkPrimaryLinkUrl'] = str(row['LinkedIn Company Page'])

    # Address fields
    if pd.notna(row.get('City')):
        data['addressAddressCity'] = str(row['City'])
    if pd.notna(row.get('State/Region')):
        data['addressAddressState'] = str(row['State/Region'])
    if pd.notna(row.get('Country/Region')):
        data['addressAddressCountry'] = str(row['Country/Region'])
    if pd.notna(row.get('Postal Code')):
        data['addressAddressPostcode'] = str(row['Postal Code'])
    if pd.notna(row.get('Street Address')):
        data['addressAddressStreet1'] = str(row['Street Address'])

    # Timestamps
    if pd.notna(row.get('Create Date')):
        data['createdAt'] = parse_timestamp(row['Create Date'])
    else:
        data['createdAt'] = datetime.utcnow().isoformat() + 'Z'

    if pd.notna(row.get('Last Modified Date')):
        data['updatedAt'] = parse_timestamp(row['Last Modified Date'])
    else:
        data['updatedAt'] = data['createdAt']

    # Map remaining HubSpot columns
    skip_cols = ['Record ID', 'Company name', 'Website URL', 'Number of Employees',
                 'LinkedIn Company Page', 'City', 'State/Region', 'Country/Region',
                 'Postal Code', 'Street Address', 'Create Date', 'Last Modified Date']

    for hubspot_col in companies_df.columns:
        if hubspot_col in skip_cols:
            continue

        pg_col_name = hubspot_col.lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        pg_col_name = ''.join(c for c in pg_col_name if c.isalnum() or c == '_')

        if pg_col_name in company_cols and pd.notna(row.get(hubspot_col)):
            target_type = company_types.get(pg_col_name, 'text')
            converted_val = convert_value_to_target_type(row[hubspot_col], target_type)
            if converted_val is not None:
                data[pg_col_name] = converted_val

    return data

def build_person_insert_data(row, cursor):
    """Build person insert data with DB-based FK resolution"""
    data = {}

    data['id'] = generate_uuid_from_hubspot_id(str(row['Record ID']))

    if pd.notna(row.get('First Name')):
        data['nameFirstName'] = str(row['First Name'])
    if pd.notna(row.get('Last Name')):
        data['nameLastName'] = str(row['Last Name'])

    if pd.notna(row.get('Email')):
        email = str(row['Email'])
        data['emailsPrimaryEmail'] = email
        # Track mapping for FK resolution in opportunities
        hubspot_contact_to_email[str(row['Record ID'])] = email

    if pd.notna(row.get('Phone Number')):
        data['phonesPrimaryPhoneNumber'] = str(row['Phone Number'])

    if pd.notna(row.get('Job Title')):
        data['jobTitle'] = str(row['Job Title'])

    if pd.notna(row.get('LinkedIn Profile')):
        data['linkedinLinkPrimaryLinkUrl'] = str(row['LinkedIn Profile'])

    if pd.notna(row.get('City')):
        data['city'] = str(row['City'])

    # Company relationship - RESOLVE BY DOMAIN (primary strategy)
    # Try both column name variations
    company_col = 'Associated Company IDs (Primary)' if 'Associated Company IDs (Primary)' in contacts_df.columns else 'Associated Company IDs'
    if pd.notna(row.get(company_col)):
        company_ids = str(row[company_col]).split(';')
        if company_ids:
            # Normalize ID: remove .0 from float representation
            company_hubspot_id = company_ids[0].strip().replace('.0', '')

            # Try to get domain from our mapping
            domain = hubspot_company_to_domain.get(company_hubspot_id)

            if domain and domain in domain_to_company_id:
                # Use actual company ID from DB (handles deduplication)
                data['companyId'] = domain_to_company_id[domain]
            else:
                # Fallback: query DB for company by domain if we have the Record ID in companies CSV
                try:
                    matching_company = companies_df[companies_df['Record ID'].astype(str) == company_hubspot_id]
                    if not matching_company.empty and pd.notna(matching_company.iloc[0].get('Website URL')):
                        lookup_domain = str(matching_company.iloc[0]['Website URL'])
                        cursor.execute(
                            f'SELECT id FROM {WORKSPACE_SCHEMA}.company WHERE "domainNamePrimaryLinkUrl" = %s LIMIT 1',
                            (lookup_domain,)
                        )
                        result = cursor.fetchone()
                        if result:
                            data['companyId'] = result[0]
                except:
                    pass  # Skip if lookup fails

    # Timestamps
    if pd.notna(row.get('Create Date')):
        data['createdAt'] = parse_timestamp(row['Create Date'])
    else:
        data['createdAt'] = datetime.utcnow().isoformat() + 'Z'

    if pd.notna(row.get('Last Modified Date')):
        data['updatedAt'] = parse_timestamp(row['Last Modified Date'])
    else:
        data['updatedAt'] = data['createdAt']

    skip_cols = ['Record ID', 'First Name', 'Last Name', 'Email', 'Phone Number',
                 'Job Title', 'LinkedIn Profile', 'City', 'Associated Company IDs', 'Associated Company IDs (Primary)',
                 'Create Date', 'Last Modified Date']

    for hubspot_col in contacts_df.columns:
        if hubspot_col in skip_cols:
            continue

        pg_col_name = hubspot_col.lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        pg_col_name = ''.join(c for c in pg_col_name if c.isalnum() or c == '_')

        if pg_col_name in person_cols and pd.notna(row.get(hubspot_col)):
            target_type = person_types.get(pg_col_name, 'text')
            converted_val = convert_value_to_target_type(row[hubspot_col], target_type)
            if converted_val is not None:
                data[pg_col_name] = converted_val

    return data

def build_opportunity_insert_data(row, cursor):
    """Build opportunity insert data with DB-based FK resolution"""
    data = {}

    data['id'] = generate_uuid_from_hubspot_id(str(row['Record ID']))
    data['name'] = str(row.get('Deal Name', '')) if pd.notna(row.get('Deal Name')) else None

    # Amount
    if pd.notna(row.get('Amount')):
        try:
            amount = float(row['Amount'])
            data['amountAmountMicros'] = int(amount * 1000000)
        except:
            pass

    if pd.notna(row.get('Deal Currency Code')):
        data['amountCurrencyCode'] = str(row['Deal Currency Code'])
    else:
        data['amountCurrencyCode'] = 'USD'

    if pd.notna(row.get('Close Date')):
        data['closeDate'] = parse_timestamp(row['Close Date'])

    if pd.notna(row.get('Deal Stage')):
        data['stage'] = map_deal_stage(str(row['Deal Stage']))
    else:
        data['stage'] = 'NEW'
    # Try both column name variations
    company_col = 'Associated Company IDs (Primary)' if 'Associated Company IDs (Primary)' in deals_df.columns else 'Associated Company IDs'
    if pd.notna(row.get(company_col)):
        company_ids = str(row[company_col]).split(';')
        if company_ids:
            # Normalize ID: remove .0 from float representation
            company_hubspot_id = company_ids[0].strip().replace('.0', '')

            # Try to get domain from our mapping
            domain = hubspot_company_to_domain.get(company_hubspot_id)

            if domain and domain in domain_to_company_id:
                # Use actual company ID from DB (handles deduplication)
                data['companyId'] = domain_to_company_id[domain]
            else:
                # Fallback: query DB for company by domain
                try:
                    matching_company = companies_df[companies_df['Record ID'].astype(str) == company_hubspot_id]
                    if not matching_company.empty and pd.notna(matching_company.iloc[0].get('Website URL')):
                        lookup_domain = str(matching_company.iloc[0]['Website URL'])
                        cursor.execute(
                            f'SELECT id FROM {WORKSPACE_SCHEMA}.company WHERE "domainNamePrimaryLinkUrl" = %s LIMIT 1',
                            (lookup_domain,)
                        )
                        result = cursor.fetchone()
                        if result:
                            data['companyId'] = result[0]
                except:
                    pass

    # Contact relationship - RESOLVE BY EMAIL (primary strategy)
    if pd.notna(row.get('Associated Contact IDs')):
        contact_ids = str(row['Associated Contact IDs']).split(';')
        if contact_ids:
            # Normalize ID: remove .0 from float representation
            contact_hubspot_id = contact_ids[0].strip().replace('.0', '')

            # Try to get email from our mapping
            email = hubspot_contact_to_email.get(contact_hubspot_id)

            if email and email in email_to_person_id:
                # Use actual person ID from DB (handles deduplication)
                data['pointOfContactId'] = email_to_person_id[email]
            else:
                # Fallback: query DB for person by email
                try:
                    matching_contact = contacts_df[contacts_df['Record ID'].astype(str) == contact_hubspot_id]
                    if not matching_contact.empty and pd.notna(matching_contact.iloc[0].get('Email')):
                        lookup_email = str(matching_contact.iloc[0]['Email'])
                        cursor.execute(
                            f'SELECT id FROM {WORKSPACE_SCHEMA}.person WHERE "emailsPrimaryEmail" = %s LIMIT 1',
                            (lookup_email,)
                        )
                        result = cursor.fetchone()
                        if result:
                            data['pointOfContactId'] = result[0]
                except:
                    pass

    # Timestamps
    if pd.notna(row.get('Create Date')):
        data['createdAt'] = parse_timestamp(row['Create Date'])
    else:
        data['createdAt'] = datetime.utcnow().isoformat() + 'Z'

    if pd.notna(row.get('Last Modified Date')):
        data['updatedAt'] = parse_timestamp(row['Last Modified Date'])
    else:
        data['updatedAt'] = data['createdAt']

    skip_cols = ['Record ID', 'Deal Name', 'Amount', 'Close Date',
                 'Deal Stage', 'Associated Company IDs', 'Associated Company IDs (Primary)',
                 'Associated Contact IDs', 'Associated Contact IDs (Primary)',
                 'Create Date', 'Last Modified Date']

    for hubspot_col in deals_df.columns:
        if hubspot_col in skip_cols:
            continue

        pg_col_name = hubspot_col.lower().replace(' ', '_').replace('/', '_').replace('-', '_')
        pg_col_name = ''.join(c for c in pg_col_name if c.isalnum() or c == '_')

        if pg_col_name in opportunity_cols and pd.notna(row.get(hubspot_col)):
            target_type = opportunity_types.get(pg_col_name, 'text')
            converted_val = convert_value_to_target_type(row[hubspot_col], target_type)
            if converted_val is not None:
                data[pg_col_name] = converted_val

    return data


print("✅ Data transformation functions loaded (with FK resolution)")

✅ Data transformation functions loaded (with FK resolution)


## 5. Execute Migration

In [31]:
def execute_migration():
    """Execute complete migration with transaction safety, per-row savepoints, and FK resolution"""
    global inserted_company_ids, inserted_person_ids
    global domain_to_company_id, email_to_person_id
    global hubspot_company_to_domain, hubspot_contact_to_email

    # Reset tracking
    inserted_company_ids = set()
    inserted_person_ids = set()
    inserted_opportunity_ids = set()
    domain_to_company_id = {}
    email_to_person_id = {}
    hubspot_company_to_domain = {}
    hubspot_contact_to_email = {}

    try:
        conn = get_db_connection(TWENTY_DB_CONFIG)
        conn.autocommit = False
        cursor = conn.cursor()

        migration_stats = {
            'companies': {'success': 0, 'failed': 0, 'errors': []},
            'persons': {'success': 0, 'failed': 0, 'errors': []},
            'opportunities': {'success': 0, 'failed': 0, 'errors': []}
        }

        # PHASE 1: Migrate Companies
        print("\n" + "="*100)
        print("PHASE 1: MIGRATING COMPANIES")
        print("="*100)
        print(f"Records to migrate: {len(companies_df)}\n")

        for idx, row in companies_df.iterrows():
            try:
                cursor.execute("SAVEPOINT sp_row")

                data = build_company_insert_data(row)
                columns = list(data.keys())
                values = [data[col] for col in columns]

                col_str = ', '.join([f'"{col}"' for col in columns])
                placeholders = ', '.join(['%s'] * len(columns))

                # Use conflict on unique domain column (index-backed)
                insert_sql = f"""
                    INSERT INTO {WORKSPACE_SCHEMA}.company ({col_str})
                    VALUES ({placeholders})
                    ON CONFLICT ("domainNamePrimaryLinkUrl") DO UPDATE SET
                        {', '.join([f'"{col}" = EXCLUDED."{col}"' for col in columns if col != 'id'])}
                    RETURNING id, "domainNamePrimaryLinkUrl"
                """

                cursor.execute(insert_sql, values)
                result = cursor.fetchone()
                if result:
                    actual_id = result[0]
                    actual_domain = result[1]

                    migration_stats['companies']['success'] += 1
                    inserted_company_ids.add(actual_id)

                    # Build FK resolution map: domain → actual company ID
                    if actual_domain:
                        domain_to_company_id[actual_domain] = actual_id

                if (idx + 1) % 100 == 0:
                    print(f"  ✅ Progress: {idx+1}/{len(companies_df)}")

            except Exception as e:
                cursor.execute("ROLLBACK TO SAVEPOINT sp_row")
                migration_stats['companies']['failed'] += 1
                error_msg = f"Row {idx}: {str(e)[:150]}"
                migration_stats['companies']['errors'].append(error_msg)
                if migration_stats['companies']['failed'] <= 5:
                    print(f"  ❌ {error_msg}")

        conn.commit()
        print(f"\n✅ Companies: {migration_stats['companies']['success']} migrated, {migration_stats['companies']['failed']} failed")
        print(f"   FK Resolution Map: {len(domain_to_company_id)} domains → company IDs")

        # PHASE 2: Migrate Contacts
        print("\n" + "="*100)
        print("PHASE 2: MIGRATING CONTACTS")
        print("="*100)
        print(f"Records to migrate: {len(contacts_df)}\n")

        for idx, row in contacts_df.iterrows():
            try:
                cursor.execute("SAVEPOINT sp_row")

                data = build_person_insert_data(row, cursor)
                columns = list(data.keys())
                values = [data[col] for col in columns]

                col_str = ', '.join([f'"{col}"' for col in columns])
                placeholders = ', '.join(['%s'] * len(columns))

                # Use conflict on unique email column (index-backed)
                insert_sql = f"""
                    INSERT INTO {WORKSPACE_SCHEMA}.person ({col_str})
                    VALUES ({placeholders})
                    ON CONFLICT ("emailsPrimaryEmail") DO UPDATE SET
                        {', '.join([f'"{col}" = EXCLUDED."{col}"' for col in columns if col != 'id'])}
                    RETURNING id, "emailsPrimaryEmail"
                """

                cursor.execute(insert_sql, values)
                result = cursor.fetchone()
                if result:
                    actual_id = result[0]
                    actual_email = result[1]

                    migration_stats['persons']['success'] += 1
                    inserted_person_ids.add(actual_id)

                    # Build FK resolution map: email → actual person ID
                    if actual_email:
                        email_to_person_id[actual_email] = actual_id

                if (idx + 1) % 100 == 0:
                    print(f"  ✅ Progress: {idx+1}/{len(contacts_df)}")

            except Exception as e:
                cursor.execute("ROLLBACK TO SAVEPOINT sp_row")
                migration_stats['persons']['failed'] += 1
                error_msg = f"Row {idx}: {str(e)[:150]}"
                migration_stats['persons']['errors'].append(error_msg)
                if migration_stats['persons']['failed'] <= 5:
                    print(f"  ❌ {error_msg}")

        conn.commit()
        print(f"\n✅ Persons: {migration_stats['persons']['success']} migrated, {migration_stats['persons']['failed']} failed")
        print(f"   FK Resolution Map: {len(email_to_person_id)} emails → person IDs")

        # PHASE 3: Migrate Deals
        print("\n" + "="*100)
        print("PHASE 3: MIGRATING DEALS")
        print("="*100)
        print(f"Records to migrate: {len(deals_df)}\n")

        for idx, row in deals_df.iterrows():
            try:
                cursor.execute("SAVEPOINT sp_row")

                data = build_opportunity_insert_data(row, cursor)
                columns = list(data.keys())
                values = [data[col] for col in columns]

                col_str = ', '.join([f'"{col}"' for col in columns])
                placeholders = ', '.join(['%s'] * len(columns))

                insert_sql = f"""
                    INSERT INTO {WORKSPACE_SCHEMA}.opportunity ({col_str})
                    VALUES ({placeholders})
                    ON CONFLICT (id) DO UPDATE SET
                        {', '.join([f'"{col}" = EXCLUDED."{col}"' for col in columns if col != 'id'])}
                """

                cursor.execute(insert_sql, values)
                migration_stats['opportunities']['success'] += 1
                inserted_opportunity_ids.add(data['id'])

                if (idx + 1) % 100 == 0:
                    print(f"  ✅ Progress: {idx+1}/{len(deals_df)}")

            except Exception as e:
                cursor.execute("ROLLBACK TO SAVEPOINT sp_row")
                migration_stats['opportunities']['failed'] += 1
                error_msg = f"Row {idx}: {str(e)[:150]}"
                migration_stats['opportunities']['errors'].append(error_msg)
                if migration_stats['opportunities']['failed'] <= 5:
                    print(f"  ❌ {error_msg}")

        conn.commit()
        print(f"\n✅ Opportunities: {migration_stats['opportunities']['success']} migrated, {migration_stats['opportunities']['failed']} failed")

        # Summary
        print("\n" + "="*100)
        print("🎉 MIGRATION COMPLETE!")
        print("="*100)

        total_success = sum([migration_stats[k]['success'] for k in migration_stats])
        total_failed = sum([migration_stats[k]['failed'] for k in migration_stats])

        print(f"""
SUMMARY:
  Companies:     {migration_stats['companies']['success']:5d} ✅  {migration_stats['companies']['failed']:5d} ❌
  Persons:       {migration_stats['persons']['success']:5d} ✅  {migration_stats['persons']['failed']:5d} ❌
  Opportunities: {migration_stats['opportunities']['success']:5d} ✅  {migration_stats['opportunities']['failed']:5d} ❌

  TOTAL:         {total_success:5d} ✅  {total_failed:5d} ❌
  Success Rate:  {(total_success/(total_success+total_failed)*100) if (total_success+total_failed) > 0 else 0:.1f}%
        """)

        cursor.close()
        conn.close()

        # return stats and ids for cleanup
        return migration_stats, {
            'company_ids': list(inserted_company_ids),
            'person_ids': list(inserted_person_ids),
            'opportunity_ids': list(inserted_opportunity_ids)
        }

    except Exception as e:
        print(f"\n❌ FATAL ERROR: {e}")
        import traceback
        traceback.print_exc()
        if 'conn' in locals() and conn:
            conn.rollback()
            conn.close()
        return None, None

# Execute the migration
print("\n🚀 STARTING MIGRATION...")
result = execute_migration()


🚀 STARTING MIGRATION...
✓ Database connection established

PHASE 1: MIGRATING COMPANIES
Records to migrate: 13


✅ Companies: 13 migrated, 0 failed
   FK Resolution Map: 13 domains → company IDs

PHASE 2: MIGRATING CONTACTS
Records to migrate: 10


✅ Persons: 10 migrated, 0 failed
   FK Resolution Map: 10 emails → person IDs

PHASE 3: MIGRATING DEALS
Records to migrate: 10


✅ Opportunities: 10 migrated, 0 failed

🎉 MIGRATION COMPLETE!

SUMMARY:
  Companies:        13 ✅      0 ❌
  Persons:          10 ✅      0 ❌
  Opportunities:    10 ✅      0 ❌

  TOTAL:            33 ✅      0 ❌
  Success Rate:  100.0%
        


## 6. Verify Migration

In [32]:
# Verify migrated data
print("="*100)
print("VERIFYING MIGRATED DATA")
print("="*100)

try:
    conn = get_db_connection(TWENTY_DB_CONFIG)
    cursor = conn.cursor()

    # Count records
    print("\n📊 Record counts:")
    print("-"*100)

    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.company")
    company_count = cursor.fetchone()[0]
    print(f"  Companies:     {company_count}")

    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.person")
    person_count = cursor.fetchone()[0]
    print(f"  Persons:       {person_count}")

    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.opportunity")
    opportunity_count = cursor.fetchone()[0]
    print(f"  Opportunities: {opportunity_count}")

    # Sample company
    print("\n\n🏢 Sample Company:")
    print("-"*100)
    cursor.execute(f"""
        SELECT id, name, "domainNamePrimaryLinkUrl", employees, "addressAddressCity", "createdAt"
        FROM {WORKSPACE_SCHEMA}.company
        WHERE name IS NOT NULL
        ORDER BY "createdAt" DESC
        LIMIT 1
    """)
    sample = cursor.fetchone()
    if sample:
        print(f"  ID: {sample[0]}")
        print(f"  Name: {sample[1]}")
        print(f"  Domain: {sample[2]}")
        print(f"  Employees: {sample[3]}")
        print(f"  City: {sample[4]}")
        print(f"  Created: {sample[5]}")

    # Sample person with company
    print("\n\n👤 Sample Person (with company):")
    print("-"*100)
    cursor.execute(f"""
        SELECT
            p.id,
            p."nameFirstName",
            p."nameLastName",
            p."emailsPrimaryEmail",
            p."companyId",
            c.name as company_name
        FROM {WORKSPACE_SCHEMA}.person p
        LEFT JOIN {WORKSPACE_SCHEMA}.company c ON p."companyId" = c.id
        WHERE p."nameFirstName" IS NOT NULL AND p."companyId" IS NOT NULL
        ORDER BY p."createdAt" DESC
        LIMIT 1
    """)
    sample = cursor.fetchone()
    if sample:
        print(f"  ID: {sample[0]}")
        print(f"  Name: {sample[1]} {sample[2]}")
        print(f"  Email: {sample[3]}")
        print(f"  Company ID: {sample[4]}")
        print(f"  Company Name: {sample[5]}")

    # Sample opportunity with relationships
    print("\n\n💼 Sample Opportunity (with relationships):")
    print("-"*100)
    cursor.execute(f"""
        SELECT
            o.id,
            o.name,
            o."amountAmountMicros",
            o."amountCurrencyCode",
            o.stage,
            c.name as company_name,
            p."nameFirstName" || ' ' || p."nameLastName" as contact_name
        FROM {WORKSPACE_SCHEMA}.opportunity o
        LEFT JOIN {WORKSPACE_SCHEMA}.company c ON o."companyId" = c.id
        LEFT JOIN {WORKSPACE_SCHEMA}.person p ON o."pointOfContactId" = p.id
        WHERE o.name IS NOT NULL
        ORDER BY o."createdAt" DESC
        LIMIT 1
    """)
    sample = cursor.fetchone()
    if sample:
        amount_display = f"${sample[2]/1000000:.2f} {sample[3]}" if sample[2] else "N/A"
        print(f"  ID: {sample[0]}")
        print(f"  Name: {sample[1]}")
        print(f"  Amount: {amount_display}")
        print(f"  Stage: {sample[4]}")
        print(f"  Company: {sample[5]}")
        print(f"  Contact: {sample[6]}")

    # Relationship stats
    print("\n\n🔗 Relationship Statistics:")
    print("-"*100)

    cursor.execute(f'SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.person WHERE "companyId" IS NOT NULL')
    persons_with_company = cursor.fetchone()[0]
    print(f"  Persons with company: {persons_with_company}/{person_count} ({persons_with_company/person_count*100:.1f}%)")

    cursor.execute(f'SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.opportunity WHERE "companyId" IS NOT NULL')
    opps_with_company = cursor.fetchone()[0]
    print(f"  Opportunities with company: {opps_with_company}/{opportunity_count} ({opps_with_company/opportunity_count*100:.1f}%)")

    cursor.execute(f'SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.opportunity WHERE "pointOfContactId" IS NOT NULL')
    opps_with_contact = cursor.fetchone()[0]
    print(f"  Opportunities with contact: {opps_with_contact}/{opportunity_count} ({opps_with_contact/opportunity_count*100:.1f}%)")

    cursor.close()
    conn.close()

    print("\n" + "="*100)
    print("✅ VERIFICATION COMPLETE!")
    print("="*100)

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()

VERIFYING MIGRATED DATA
✓ Database connection established

📊 Record counts:
----------------------------------------------------------------------------------------------------
  Companies:     14
  Persons:       10
  Opportunities: 12


🏢 Sample Company:
----------------------------------------------------------------------------------------------------
  ID: c28e47b5-f09d-5867-bb9d-de0ea8e924f9
  Name: Stadt Karlsruhe
  Domain: wifoe.karlsruhe.de
  Employees: None
  City: None
  Created: 2025-10-10 08:11:00+00:00


👤 Sample Person (with company):
----------------------------------------------------------------------------------------------------
  ID: fdf01828-b6f8-57f7-9c75-c40baf6a8ef5
  Name: Dario Arndt
  Email: arndt@luebeck.org
  Company ID: d97685a9-3420-5313-a01b-52011550eab6
  Company Name: Wirtschaftsförderung Lübeck


💼 Sample Opportunity (with relationships):
----------------------------------------------------------------------------------------------------
  ID: a5360e

## 7. 🧹 Cleanup - Purge Test Data (Optional)

# ⚠️  DANGER ZONE: Purge all migrated data from tables
# Use this cell to clean up test data before running another migration
print("\n🧹 PURGING ALL DATA FROM TABLES...")
print("="*100)
print("⚠️  WARNING: This will delete ALL records from company, person, and opportunity tables!")
print("="*100)

try:
    conn = get_db_connection(TWENTY_DB_CONFIG)
    cursor = conn.cursor()

    # Delete in correct order (FK constraints)
    print("\nDeleting opportunities...")
    cursor.execute(f"DELETE FROM {WORKSPACE_SCHEMA}.opportunity")
    opp_deleted = cursor.rowcount

    print("Deleting persons...")
    cursor.execute(f"DELETE FROM {WORKSPACE_SCHEMA}.person")
    person_deleted = cursor.rowcount

    print("Deleting companies...")
    cursor.execute(f"DELETE FROM {WORKSPACE_SCHEMA}.company")
    company_deleted = cursor.rowcount

    conn.commit()

    # Verify empty tables
    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.company")
    c_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.person")
    p_count = cursor.fetchone()[0]
    cursor.execute(f"SELECT COUNT(*) FROM {WORKSPACE_SCHEMA}.opportunity")
    o_count = cursor.fetchone()[0]

    cursor.close()
    conn.close()

    print(f"\n✅ PURGE COMPLETE:")
    print(f"   Companies deleted:     {company_deleted} (remaining: {c_count})")
    print(f"   Persons deleted:       {person_deleted} (remaining: {p_count})")
    print(f"   Opportunities deleted: {opp_deleted} (remaining: {o_count})")
    print("="*100)
    print("✅ All tables are now empty and ready for a fresh migration!")
    print("="*100)

except Exception as e:
    print(f"\n❌ Error: {e}")
    import traceback
    traceback.print_exc()